<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/pipeline_traduz_musica_ingles_para_portugues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de Tradução de Música com Whisper

Instruções para criar um pipeline completo de tradução de música do inglês para o português usando a biblioteca Whisper.

## instrucao1 - Instalação da biblioteca Whisper

```bash
!pip install -q git+https://github.com/openai/whisper.git
```

## instrucao2 - Instalação do programa ffmpeg

```bash
!apt-get install -y ffmpeg
```

## instrucao3 - Download do arquivo .mp4

```bash
!wget https://github.com/armandossrecife/teste/raw/refs/heads/main/Adrienne.mp4
```

## instrucao4 - Transcrição do áudio

```python
import whisper

audio_path = "/content/Adrienne.mp4"

print(f"Arquivo: {audio_path}")

# Modelos possíveis: tiny, base, small, medium, large
# "small" é rápido e bom para português, "large" é mais preciso mas mais lento
modelo_selecionado = "small"
print(f"Carregando o modelo: {modelo_selecionado}")
model = whisper.load_model(modelo_selecionado)
print("Modelo carregado com sucesso!")

try:
  print(f"Iniciando a transcrição do áudio {audio_path}")
  # Transcrever
  result = model.transcribe(audio_path, language="pt")
  print("\n--- TRANSCRIÇÃO COMPLETA ---\n")
  print(result["text"])

  print(f"Salvando o texto...")
  # Salvar transcrição em arquivo TXT
  nome_arquivo_texto = "transcricao.txt"
  with open(nome_arquivo_texto, "w", encoding="utf-8") as f:
    f.write(result["text"])
  print("Texto salvo com sucesso!")
except Exception as ex:
  print(f"Erro {str(ex)} ao fazer a transcrição do arquivo {audio_path}")
```

## instrucao5 - Engenharia do Prompt de tradução

```python
# Carregar a transcrição
with open("transcricao.txt", "r", encoding="utf-8") as f:
    texto_original = f.read()

# Prompt para tradução poética
prompt_traducao = f"""
Você pode traduzir o seguinte texto de inglês para português?
Além disso, você pode deixar essa tradução em um formato mais poético/musical
(para soar como letra de música em português)?

Texto para traduzir:
{texto_original}
"""

print("Prompt de tradução criado com sucesso!")
print(prompt_traducao[:500] + "...")  # Mostrar apenas parte do prompt
```

## instrucao6 - Salvar a tradução em JSON

```python
import json

# Supondo que temos a tradução (aqui seria substituído pela resposta real da API)
traducao_pt = {
    "titulo": "Adrienne (Tradução Poética)",
    "letra_original": texto_original,
    "letra_traduzida": "[Aqui viria a tradução poética gerada]",
    "artista": "Artista Original",
    "tradutor": "Whisper + OpenAI",
    "data_traducao": "2023-11-10"
}

# Salvar em arquivo JSON
with open("traducao_musica.json", "w", encoding="utf-8") as f:
    json.dump(traducao_pt, f, ensure_ascii=False, indent=2)

print("Tradução salva em traducao_musica.json")
```

## instrucao7 - Explicação da tradução

```python
# Carregar a tradução
with open("traducao_musica.json", "r", encoding="utf-8") as f:
    traducao = json.load(f)

# Prompt para explicação da tradução
prompt_explicacao = f"""
Agora, uma vez o texto traduzido, você pode explicar o significado dessa tradução em português?

Texto original:
{traducao['letra_original']}

Tradução para português:
{traducao['letra_traduzida']}

Por favor, explique as escolhas de tradução, o significado da música e como a versão poética em português captura a essência da original.
"""

print("Prompt para explicação da tradução criado com sucesso!")
print(prompt_explicacao[:500] + "...")  # Mostrar apenas parte do prompt
```

## Pipeline Completo

Este pipeline completo:
1. Instala as dependências necessárias
2. Baixa um arquivo de áudio de exemplo
3. Transcreve o áudio usando o modelo Whisper
4. Prepara um prompt para tradução poética
5. Salva a tradução em formato JSON
6. Gera uma explicação sobre as escolhas de tradução

Nota: Para completar totalmente o pipeline, seria necessário integrar com uma API de IA (como a API da OpenAI) para processar os prompts de tradução e explicação. O código acima prepara todos os elementos, mas a execução completa dependeria dessa integração adicional.

# Implementação do Pipeline usando a API do Gemini

In [1]:
# instrucao1 - Instalar a biblioteca whisper
!pip install -q git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# instrucao2 - Instalar o programa ffmpeg
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
# instrucao3 - Baixar um arquivo .mp4 público
!wget https://github.com/armandossrecife/teste/raw/refs/heads/main/Adrienne.mp4

--2025-08-22 02:36:26--  https://github.com/armandossrecife/teste/raw/refs/heads/main/Adrienne.mp4
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/armandossrecife/teste/refs/heads/main/Adrienne.mp4 [following]
--2025-08-22 02:36:26--  https://raw.githubusercontent.com/armandossrecife/teste/refs/heads/main/Adrienne.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14944332 (14M) [application/octet-stream]
Saving to: ‘Adrienne.mp4’

Adrienne.mp4        100%[===================>]  14.25M  --.-KB/s    in 0.1s    

2025-08-22 02:36:27 (113 MB/s) - ‘Adrienne.mp4’ saved [14944332/14944332]



In [4]:
# Instalar a biblioteca do Google Gemini
!pip install -q google-generativeai

In [8]:
import whisper
import google.generativeai as genai
import json
import os

# Configurar a API do Google Gemini
# Você precisa ter uma chave de API válida
GEMINI_API_KEY = "?"  # Substitua pela sua chave API
genai.configure(api_key=GEMINI_API_KEY)

# Configurar o modelo Gemini
modelo_gemini = genai.GenerativeModel('gemini-2.0-flash')

In [9]:
# instrucao4 - Transcrição do áudio com Whisper
def transcrever_audio(audio_path):
    print(f"Arquivo: {audio_path}")

    # Modelos possíveis: tiny, base, small, medium, large
    modelo_selecionado = "small"
    print(f"Carregando o modelo: {modelo_selecionado}")
    model = whisper.load_model(modelo_selecionado)
    print("Modelo carregado com sucesso!")

    try:
        print(f"Iniciando a transcrição do áudio {audio_path}")
        # Transcrever
        result = model.transcribe(audio_path, language="pt")
        print("\n--- TRANSCRIÇÃO COMPLETA ---\n")
        print(result["text"])

        print(f"Salvando o texto...")
        # Salvar transcrição em arquivo TXT
        nome_arquivo_texto = "transcricao.txt"
        with open(nome_arquivo_texto, "w", encoding="utf-8") as f:
            f.write(result["text"])
        print("Texto salvo com sucesso!")

        return result["text"]
    except Exception as ex:
        print(f"Erro {str(ex)} ao fazer a transcrição do arquivo {audio_path}")
        return None

# instrucao5 - Tradução com Gemini
def traduzir_texto(texto_ingles):
    prompt_traducao = f"""
    Você pode traduzir o seguinte texto de inglês para português?
    Além disso, você pode deixar essa tradução em um formato mais poético/musical
    (para soar como letra de música em português)?

    Texto para traduzir:
    {texto_ingles}
    """

    try:
        response = modelo_gemini.generate_content(prompt_traducao)
        return response.text
    except Exception as e:
        print(f"Erro ao traduzir: {e}")
        return None

# instrucao6 - Salvar em JSON
def salvar_traducao_json(texto_original, texto_traduzido, nome_arquivo="traducao_musica.json"):
    dados_traducao = {
        "original": texto_original,
        "traducao_poetica": texto_traduzido,
        "idioma_original": "inglês",
        "idioma_traducao": "português"
    }

    with open(nome_arquivo, "w", encoding="utf-8") as f:
        json.dump(dados_traducao, f, ensure_ascii=False, indent=2)

    print(f"Tradução salva em {nome_arquivo}")

# instrucao7 - Explicação da tradução
def explicar_traducao(texto_original, texto_traduzido):
    prompt_explicacao = f"""
    Agora, uma vez o texto traduzido, você pode explicar o significado dessa tradução em português?

    Texto original em inglês:
    {texto_original}

    Tradução para português:
    {texto_traduzido}

    Por favor, explique as escolhas de tradução, o significado da música e como a versão poética em português
    captura a essência da original. Escreva em português.
    """

    try:
        response = modelo_gemini.generate_content(prompt_explicacao)

        # Salvar explicação em arquivo
        with open("explicacao_traducao.txt", "w", encoding="utf-8") as f:
            f.write(response.text)

        print("Explicação da tradução salva em explicacao_traducao.txt")
        return response.text
    except Exception as e:
        print(f"Erro ao gerar explicação: {e}")
        return None

# Função principal para executar o pipeline completo
def pipeline_traducao_musica(audio_path):
    print("=== INICIANDO PIPELINE DE TRADUÇÃO DE MÚSICA ===\n")

    # Passo 1: Transcrever áudio
    print("1. Transcrevendo áudio...")
    texto_original = transcrever_audio(audio_path)

    if texto_original is None:
        print("Falha na transcrição. Abortando pipeline.")
        return

    # Passo 2: Traduzir texto
    print("\n2. Traduzindo texto...")
    texto_traduzido = traduzir_texto(texto_original)

    if texto_traduzido is None:
        print("Falha na tradução. Abortando pipeline.")
        return

    print("\n--- TRADUÇÃO POÉTICA ---\n")
    print(texto_traduzido)

    # Passo 3: Salvar em JSON
    print("\n3. Salvando tradução em JSON...")
    salvar_traducao_json(texto_original, texto_traduzido)

    # Passo 4: Explicar tradução
    print("\n4. Gerando explicação da tradução...")
    explicacao = explicar_traducao(texto_original, texto_traduzido)

    if explicacao:
        print("\n--- EXPLICAÇÃO DA TRADUÇÃO ---\n")
        print(explicacao)

    print("\n=== PIPELINE CONCLUÍDO COM SUCESSO ===")

In [10]:
audio_path = "/content/Adrienne.mp4"

# Verificar se o arquivo de áudio existe
if not os.path.exists(audio_path):
  print(f"Arquivo {audio_path} não encontrado. Verifique o caminho.")
else:
  pipeline_traducao_musica(audio_path)

=== INICIANDO PIPELINE DE TRADUÇÃO DE MÚSICA ===

1. Transcrevendo áudio...
Arquivo: /content/Adrienne.mp4
Carregando o modelo: small
Modelo carregado com sucesso!
Iniciando a transcrição do áudio /content/Adrienne.mp4


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



--- TRANSCRIÇÃO COMPLETA ---

 🎵 🎵 I've been thinking about you, my love 🎵 🎵 And all the crazy things that you've brought me through 🎵 🎵 And I've been coming around 🎵 Throwing it back to you What you thinkin' of me? When you kiss him Could you tease me when you lick the skin? And all the while I showered you with trust and promises What I'm needin' now, some sweet revenge To get back on that I lost then To leave you all I had to give But I could never reach him Adia I thought I knew you once again You used me, used me Adia I should have lived you long before You used me, used me Oh I used to have my money, drove my car I treated you like a shining star But in my sky, all burnt out you I'll have the last laugh When I see you walkin' some other guy Cause I know you're gonna end up all alone So take these words, some good advice All you've done's could come back twice Never cared how much it hurt I really need to tell you Adia I thought I knew you once again You used me, used me Adia I s